In [2]:
#from PIL import Image
import facebook
#import pytesseract
import requests
# import html
from bs4 import BeautifulSoup
from moviepy.editor import *
import pandas as pd
# from googletrans import Translator
import speech_recognition as sr
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from datetime import date
from selenium import webdriver

In [3]:
df = pd.read_csv('FacebookAds/CT.csv')

In [4]:
df.head()

,Unnamed: 0,ad_creative_bodies,ad_creative_link_descriptions,bylines,ad_creative_link_titles,ad_creative_link_captions,demographic_distribution,page_name,page_id,id,estimated_audience_size.lower_bound,estimated_audience_size.upper_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound
0,0,"[""Please join CEA's Red for Ed Day of Action t...","[""With only nine days left in the legislative ...",CONNECTICUT EDUCATION ASSOCIATION,['Join us in Hartford or from Your School for ...,['cea.org'],"[{'percentage': '0.000954', 'age': '65+', 'gen...",Connecticut Education Association (CEA),124383441363,688988779084177,100001.0,500000.0,10000,14999,200,299
1,1,['Thinking about attending a Connecticut State...,NaN,odonnell company,NaN,NaN,"[{'percentage': '0.000541', 'age': '13-17', 'g...",Connecticut State Community Colleges,406907476563016,247626834204512,500001.0,1000000.0,2000,2999,100,199
2,2,['Governor Lamont is protecting abortion right...,['Gov. Lamont to sign key abortion bill'],NED FOR CT,['Sign up for more updates'],['nedlamont.com'],"[{'percentage': '0.018349', 'age': '18-24', 'g...",Ned Lamont,196395247518,1318295712031128,500001.0,1000000.0,0,999,0,99
3,3,"['Hey Connecticut, have you heard? Governor La...",['Gov. Lamont suspends gas tax'],NED FOR CT,['Sign up for more updates'],['nedlamont.com'],NaN,Ned Lamont,196395247518,1072615439989473,500001.0,1000000.0,0,999,0,99
4,4,['Cost increases are hitting families in Conne...,['Sign up now >>'],NED FOR CT,['Gov. Lamont suspends gas tax'],['nedlamont.com'],NaN,Ned Lamont,196395247518,667887484316111,500001.0,1000000.0,0,999,0,99


In [11]:
df = df[df.id == 356033606584492]
df

,Unnamed: 0,ad_creative_bodies,ad_creative_link_descriptions,bylines,ad_creative_link_titles,ad_creative_link_captions,demographic_distribution,page_name,page_id,id,estimated_audience_size.lower_bound,estimated_audience_size.upper_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound
334,334,['Paul Crisci – a name you’ll soon be hearing ...,['No Politics. Just Common Sense.'],Paul For Senate,['The Kind of Candidate Connecticut Needs.'],['pcforct.com'],"[{'percentage': '0.000793', 'age': '18-24', 'g...",Paul Crisci for State Senate,111019954865337,356033606584492,50001.0,100000.0,7000,7999,0,99


In [33]:
r = sr.Recognizer()

In [36]:
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 150,
        # adjust this per requirement
        silence_thresh = sound.dBFS-4,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [27]:
video = VideoFileClip(vidURL)
audio = video.audio
audio.write_audiofile('audio.wav')

MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [39]:
vidURL = ''
for index, row in df.iterrows():
    # myDict = {'q':str(row['id']),'access_token':nuToken,'active_status':'all','ad_type','political_and_issue_ads'}
    myDict = {'id':str(row['id'])}
    print(row['id'])
    driver = webdriver.Firefox()
    driver.get(f'https://www.facebook.com/ads/library/?id={row["id"]}')

#     profile = graph.get_object('ads_archive',ad_active_status='all',ad_delivery_date_max=today,ad_delivery_date_min='2019-06-01',fields='ad_snapshot_url',search_terms="''",ad_type='POLITICAL_AND_ISSUE_ADS',ad_reached_countries=['US'],limit=25)
#     header = {'access_token': nuToken}
    # url = 'https://www.facebook.com/ads/library/?'
    # response = requests.get(url, params=myDict)
    #print(response.status_code)
    html = driver.page_source
    soup = BeautifulSoup(html)
    # soup = BeautifulSoup(response.text)
    vidURL = soup.find('video')['src']
    driver.close()
    print(vidURL)
    video = VideoFileClip(vidURL)
    audio = video.audio
    audio.write_audiofile('audio.wav')
    text_to_check = get_large_audio_transcription('audio.wav')
    print(text_to_check)
    #break

356033606584492
https://video-lax3-1.xx.fbcdn.net/v/t42.1790-2/278405859_356033633251156_1091817099657037947_n.?_nc_cat=105&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=O8-YzmxhwkEAX_Ko2JU&_nc_ht=video-lax3-1.xx&oh=00_AT8nv5fESP3mPh-N_SQOP52WA57otuDatOLbwWpQNkxM_w&oe=629824FE
MoviePy - Writing audio in audio.wav


MoviePy - Done.
audio-chunks/chunk1.wav : Who's. 
audio-chunks/chunk2.wav : Chrissy. 
audio-chunks/chunk3.wav : And why should i care. 
audio-chunks/chunk4.wav : Years ago i was working on. 
Error: 
audio-chunks/chunk6.wav : Prodigy. 
audio-chunks/chunk7.wav : After a few months. 
Error: 
audio-chunks/chunk9.wav : Camping mattresses. 
audio-chunks/chunk10.wav : Have you ever thought about running. 
audio-chunks/chunk11.wav : Politics. 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
audio-chunks/chunk18.wav : Campaign. 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
audio-chunks/chunk28.wav : Call leticia. 
Error: 
Error: 
Error: 
audio-chunks/chunk32.wav : Zach. 
audio-chunks/chunk33.wav : Connecticut. 
audio-chunks/chunk34.wav : Well i'm not a politician. 
Error: 
audio-chunks/chunk36.wav : Leaving. 
audio-chunks/chunk37.wav : Promise. 
audio-chunks/chunk38.wav : And thomas november. 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Err